In [14]:
import pandas as pd
import sasoptpy as so
import re
import os
import subprocess
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [15]:
t0 = time.time()

In [16]:
df = pd.read_csv('../data/input.csv')
gk_data = df[df['Pos'] == 'G'].copy().reset_index()
gk_data.set_index('index', inplace=True)

In [17]:
def find_optimal_goalkeepers(gk_data, budget):
    # Initializing the model
    model = so.Model(name='gk_model')
    players = gk_data.index.tolist()
    
    # Variables
    lineup = model.add_variables(players, name='lineup', vartype=so.binary)
    bench = model.add_variables(players, name='bench', vartype=so.binary)
    
    # Objective
    total_xp = so.expr_sum(lineup[p] * gk_data.loc[p, '3_Pts'] for p in players) + 0.1 * so.expr_sum(bench[p] * gk_data.loc[p, '3_Pts'] for p in players)
    model.set_objective(-total_xp, name='total_xp_obj', sense='N')
    
    # Constraints
    model.add_constraints((lineup[p] + bench[p] <= 1 for p in players), name='lineup_or_bench')
    model.add_constraint(so.expr_sum(lineup[p] for p in players) == 1, name='single_lineup')
    model.add_constraint(so.expr_sum(bench[p] for p in players) == 1, name='single_bench')
    model.add_constraint(so.expr_sum((lineup[p] + bench[p]) * gk_data.loc[p, 'BV'] for p in players) <= budget, name='budget_constraint')
    
    # Solution
    model.export_mps(filename='gk.mps')
    command = 'cbc gk.mps solve solu solution.txt'
    # !{command}
    #os.system(command)
    subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Parse solution
    with open('solution.txt', 'r') as f:
        for v in model.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            var = model.get_variable(words[1])
            var.set_value(float(words[2]))

    # Print results
    print("LINEUP")
    for p in players:
        if lineup[p].get_value() > 0.5:
            print(p, gk_data.loc[p])
    
    print("BENCH")
    for p in players:
        if bench[p].get_value() > 0.5:
            print(p, gk_data.loc[p])

In [18]:
find_optimal_goalkeepers(gk_data, 9.5)

NOTE: Initialized model gk_model.
LINEUP
46 Pos                   G
ID                   47
Name           Martinez
BV                  5.0
SV                  5.0
Team        Aston Villa
3_xMins              94
3_Pts              3.82
4_xMins              93
4_Pts              3.88
5_xMins              92
5_Pts              3.84
6_xMins              93
6_Pts              3.74
7_xMins              92
7_Pts              3.45
8_xMins              91
8_Pts               3.5
9_xMins              92
9_Pts              3.64
10_xMins             91
10_Pts             3.14
Elite%              0.0
Name: 46, dtype: object
BENCH
90 Pos                 G
ID                 91
Name          Flekken
BV                4.5
SV                4.5
Team        Brentford
3_xMins            94
3_Pts            3.78
4_xMins            93
4_Pts            3.06
5_xMins            93
5_Pts            3.09
6_xMins            92
6_Pts            3.41
7_xMins            92
7_Pts            3.62
8_xMins            

In [6]:
t1 = time.time()
print(t1-t0, 'seconds')

0.1166079044342041 seconds
